In [2]:
import numpy as np 
import pandas as pd 
import librosa 
from glob import glob
import os
from scipy.stats import skew, kurtosis

In [3]:
data = pd.read_csv('/home/yamen/Projects/Technocolabs/extracted_chorus.csv')
df = data.copy()
df

,Artist,Title,Label,Path,Chorus_Path,extracted_chorus_path
0,The Weeknd,Blinding Lights,1,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/Blinding Lights - The Weeknd.wav
1,Olivia Rodrigo,Good 4 U,1,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/Good 4 U - Olivia Rodrigo.wav
2,Olivia Rodrigo,Drivers License,1,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/Drivers License - Olivia Rodrig...
3,Lil Nas X,Montero (Call Me By Your Name),1,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/Montero (Call Me By Your Name) ...
4,BTS,Butter,1,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/Butter - BTS.wav
...,...,...,...,...,...,...
790,Ne-Yo,Do You,0,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/Do You - Ne-Yo.wav
791,Avril Lavigne,When You're Gone,0,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/When You're Gone - Avril Lavign...
792,Gwen Stefani,4 In The Morning,0,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/4 In The Morning - Gwen Stefani...
793,Beyonce,Get Me Bodied,0,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/Get Me Bodied - Beyonce.wav


In [9]:
paths = list(df['Path']) # list for extracted chorus paths
titles = list(df['Title']) # list of song titles
data = [] # list to contain extracted audio features 

In [10]:
# helper function to get statistics of each dimension data for each audio raw feature 
def statistics(list, feature, columns_name, data):
    i = 0 # label to track the number of raw dimensions for each audio feature 
    for ele in list:
        # get the 7 statististics of each raw dimension
        _skew = skew(ele)
        columns_name.append(f'{feature}_kew_{i}')
        min = np.min(ele)
        columns_name.append(f'{feature}_min_{i}')
        max = np.max(ele)
        columns_name.append(f'{feature}_max_{i}')
        std = np.std(ele)
        columns_name.append(f'{feature}_std_{i}')
        mean = np.mean(ele)
        columns_name.append(f'{feature}_mean_{i}')
        median = np.median(ele)
        columns_name.append(f'{feature}_median_{i}')
        _kurtosis = kurtosis(ele)
        columns_name.append(f'{feature}_kurtosis_{i}')

        i += 1
        # append 7 statistics as single row in list 
        data.append(_skew) 
        data.append(min)
        data.append(max) 
        data.append(std) 
        data.append(mean) 
        data.append(median) 
        data.append(_kurtosis)
    
    return data

In [12]:
# helper function ectract audio feature with statistics 
def extract_features(audio_path, title):

    data = []
    columns_name = ['title']
    data.append(title)
    x , sr = librosa.load(audio_path) #load audio waveform and sample rate

    chroma_stft = librosa.feature.chroma_stft(x, sr) # chroma_stft with 12 raw dims
    statistics(chroma_stft, 'chroma_stft', columns_name, data) # stats = 12 dims * 7 stats data

    chroma_cqt = librosa.feature.chroma_cqt(x, sr) # chroma_cqt with 12 raw dims
    statistics(chroma_cqt, 'chroma_cqt', columns_name, data) # stats = 12 dims * 7 stats data

    chroma_cens = librosa.feature.chroma_cens(x, sr) # chroma_cens with 12 raw dims
    statistics(chroma_cens, 'chroma_cens', columns_name, data) # stats = 12 dims * 7 stats data

    mfcc = librosa.feature.mfcc(x, sr) # mfcc with 20 raw dims
    statistics(mfcc, 'mfcc', columns_name, data)  # stats = 20 dims * 7 stats data
    
    rms = librosa.feature.rms(x, sr) # rms with 1 raw dims
    statistics(rms, 'rms', columns_name, data) # stats = 1 dims * 7 stats data

    spectral_centroid = librosa.feature.spectral_centroid(x , sr) # spectral_centroid with 1 raw dims
    statistics(spectral_centroid, 'spectral_centroid', columns_name, data) # stats = 1 dims * 7 stats data

    spectral_bandwidth = librosa.feature.spectral_bandwidth(x , sr) # spectral_bandwidth with 1 raw dims
    statistics(spectral_bandwidth, 'spectral_bandwidth', columns_name, data) # stats = 1 dims * 7 stats data

    spectral_contrast = librosa.feature.spectral_contrast(x , sr) # spectral_contrast with 7 raw dims
    statistics(spectral_contrast, 'spectral_contrast', columns_name, data) # stats = 7 dims * 7 stats data
    
    spectral_rolloff = librosa.feature.spectral_rolloff(x , sr) # spectral_rolloff with 1 raw dims
    statistics(spectral_rolloff, 'spectral_rolloff', columns_name, data) # stats = 1 dims * 7 stats data

    tonnetz = librosa.feature.tonnetz(x , sr) # tonnetz with 6 raw dims
    statistics(tonnetz, 'tonnetz', columns_name, data) # stats = 6 dims * 7 stats data

    zero_crossing_rate = librosa.feature.zero_crossing_rate(x , sr) # zero_crossing_rate with 1 raw dims
    statistics(zero_crossing_rate, 'zero_crossing_rate', columns_name, data) # stats = 1 dims * 7 stats data

    # total number of expected dimensions - 518

    return data, columns_name

In [13]:
# extract audio freatures for all songs choruses/Hook
i = 0
while i < len(paths):
    audio_path = paths[i]
    d, cols = extract_features(audio_path, titles[i])
    data.append(d)
    print(f'The {i} song Done...')
    i += 1

/tmp/ipykernel_20026/404159168.py:9: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_stft = librosa.feature.chroma_stft(x, sr) # chroma_stft with 12 raw dims
/tmp/ipykernel_20026/404159168.py:12: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_cqt = librosa.feature.chroma_cqt(x, sr) # chroma_cqt with 12 raw dims
/tmp/ipykernel_20026/404159168.py:15: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_cens = librosa.feature.chroma_cens(x, sr) # chroma_cens with 12 raw dims
/tmp/ipykernel_20026/404159168.py:18: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an 

The 0 song Done...


/home/yamen/.local/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/tmp/ipykernel_20026/404159168.py:9: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_stft = librosa.feature.chroma_stft(x, sr) # chroma_stft with 12 raw dims
/tmp/ipykernel_20026/404159168.py:12: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_cqt = librosa.feature.chroma_cqt(x, sr) # chroma_cqt with 12 raw dims


The 1 song Done...
The 2 song Done...
The 3 song Done...
The 4 song Done...
The 5 song Done...
The 6 song Done...
The 7 song Done...
The 8 song Done...
The 9 song Done...
The 10 song Done...
The 11 song Done...
The 12 song Done...
The 13 song Done...
The 14 song Done...
The 15 song Done...
The 16 song Done...
The 17 song Done...
The 18 song Done...
The 19 song Done...


/tmp/ipykernel_20026/404159168.py:9: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  6.3144519e-08
 -1.2040135e-07  0.0000000e+00], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_stft = librosa.feature.chroma_stft(x, sr) # chroma_stft with 12 raw dims
/tmp/ipykernel_20026/404159168.py:12: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  6.3144519e-08
 -1.2040135e-07  0.0000000e+00], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_cqt = librosa.feature.chroma_cqt(x, sr) # chroma_cqt with 12 raw dims
/tmp/ipykernel_20026/404159168.py:15: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  6.3144519e-08
 -1.2040135e-07  0.0000000e+00], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_cens = librosa.feature.chroma_cens(x, s

...

The 784 song Done...
The 785 song Done...
The 786 song Done...
The 787 song Done...
The 788 song Done...
The 789 song Done...
The 790 song Done...


/tmp/ipykernel_20026/404159168.py:9: FutureWarning: Pass y=[0.000000e+00 0.000000e+00 0.000000e+00 ... 5.862699e-05 5.119421e-05
 0.000000e+00], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_stft = librosa.feature.chroma_stft(x, sr) # chroma_stft with 12 raw dims
/tmp/ipykernel_20026/404159168.py:12: FutureWarning: Pass y=[0.000000e+00 0.000000e+00 0.000000e+00 ... 5.862699e-05 5.119421e-05
 0.000000e+00], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_cqt = librosa.feature.chroma_cqt(x, sr) # chroma_cqt with 12 raw dims
/tmp/ipykernel_20026/404159168.py:15: FutureWarning: Pass y=[0.000000e+00 0.000000e+00 0.000000e+00 ... 5.862699e-05 5.119421e-05
 0.000000e+00], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_cens = librosa.feature.chroma_cens(x, sr) # chroma_cens with 12 raw dims
/t

The 791 song Done...
The 792 song Done...


/tmp/ipykernel_20026/404159168.py:9: FutureWarning: Pass y=[0.         0.         0.         ... 0.0001529  0.00018151 0.        ], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_stft = librosa.feature.chroma_stft(x, sr) # chroma_stft with 12 raw dims
/tmp/ipykernel_20026/404159168.py:12: FutureWarning: Pass y=[0.         0.         0.         ... 0.0001529  0.00018151 0.        ], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_cqt = librosa.feature.chroma_cqt(x, sr) # chroma_cqt with 12 raw dims
/tmp/ipykernel_20026/404159168.py:15: FutureWarning: Pass y=[0.         0.         0.         ... 0.0001529  0.00018151 0.        ], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_cens = librosa.feature.chroma_cens(x, sr) # chroma_cens with 12 raw dims
/tmp/ipykernel_20026/404159168.py:18: Fut

The 793 song Done...
The 794 song Done...


In [14]:
newData = pd.DataFrame(data, columns=cols) # create dataframe for new extracted audio features 
newData

,title,chroma_stft_kew_0,chroma_stft_min_0,chroma_stft_max_0,chroma_stft_std_0,chroma_stft_mean_0,chroma_stft_median_0,chroma_stft_kurtosis_0,chroma_stft_kew_1,chroma_stft_min_1,...,tonnetz_mean_5,tonnetz_median_5,tonnetz_kurtosis_5,zero_crossing_rate_kew_0,zero_crossing_rate_min_0,zero_crossing_rate_max_0,zero_crossing_rate_std_0,zero_crossing_rate_mean_0,zero_crossing_rate_median_0,zero_crossing_rate_kurtosis_0
0,Blinding Lights,0.325105,0.000000,1.0,0.318647,0.482954,0.427972,-1.184744,0.603670,0.000000,...,0.009015,0.001993,0.683537,0.120783,0.000000,0.185714,0.031305,0.075716,0.076916,0.640352
1,Good 4 U,1.020729,0.000000,1.0,0.267815,0.339303,0.279227,0.270703,0.687065,0.000000,...,-0.001283,-0.001245,2.368123,-0.139140,0.000000,0.274331,0.042506,0.089507,0.096644,0.533097
2,Drivers License,1.408063,0.000000,1.0,0.297368,0.257008,0.128317,0.812924,1.373638,0.000000,...,-0.014449,-0.014515,0.351270,1.100266,0.000000,0.248435,0.030082,0.069129,0.065079,3.259196
3,Montero (Call Me By Your Name),0.995654,0.000000,1.0,0.252682,0.311545,0.255675,0.407962,0.787342,0.000000,...,-0.001599,-0.007845,0.446242,0.525206,0.000000,0.362494,0.052102,0.114976,0.109138,0.934361
4,Butter,0.314336,0.000000,1.0,0.340640,0.459596,0.394586,-1.262645,0.618538,0.000000,...,0.004915,0.003381,4.878381,0.268373,0.000000,0.250340,0.044437,0.092644,0.090476,1.093109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790,Do You,0.649463,0.000000,1.0,0.316835,0.404342,0.315489,-0.854431,0.636219,0.000000,...,-0.025344,-0.025509,1.378500,1.289102,0.033016,0.303900,0.027651,0.096831,0.094558,5.633785
791,When You're Gone,1.357484,0.001466,1.0,0.267491,0.282992,0.193882,1.023286,1.142590,0.000649,...,0.009152,0.005985,0.361181,3.448056,0.027120,0.538277,0.047306,0.116857,0.119138,27.333917
792,4 In The Morning,0.520500,0.000000,1.0,0.344282,0.443352,0.337619,-1.191924,0.927015,0.000000,...,0.008184,-0.005062,1.474136,0.772716,0.000000,0.279184,0.036193,0.105221,0.102313,2.393584
793,Get Me Bodied,0.865777,0.000000,1.0,0.272817,0.373551,0.283081,-0.263668,0.884234,0.000000,...,0.008661,-0.000451,1.255475,0.859274,0.021088,0.393424,0.059574,0.118995,0.111610,1.050555


In [15]:
df # previous data table

,Artist,Title,Label,Path,Chorus_Path,extracted_chorus_path
0,The Weeknd,Blinding Lights,1,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/Blinding Lights - The Weeknd.wav
1,Olivia Rodrigo,Good 4 U,1,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/Good 4 U - Olivia Rodrigo.wav
2,Olivia Rodrigo,Drivers License,1,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/Drivers License - Olivia Rodrig...
3,Lil Nas X,Montero (Call Me By Your Name),1,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/Montero (Call Me By Your Name) ...
4,BTS,Butter,1,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/Butter - BTS.wav
...,...,...,...,...,...,...
790,Ne-Yo,Do You,0,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/Do You - Ne-Yo.wav
791,Avril Lavigne,When You're Gone,0,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/When You're Gone - Avril Lavign...
792,Gwen Stefani,4 In The Morning,0,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/4 In The Morning - Gwen Stefani...
793,Beyonce,Get Me Bodied,0,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/Get Me Bodied - Beyonce.wav


In [16]:
final_data = df.join(newData) # join both datasets of same number of rows 
final_data

,Artist,Title,Label,Path,Chorus_Path,extracted_chorus_path,title,chroma_stft_kew_0,chroma_stft_min_0,chroma_stft_max_0,...,tonnetz_mean_5,tonnetz_median_5,tonnetz_kurtosis_5,zero_crossing_rate_kew_0,zero_crossing_rate_min_0,zero_crossing_rate_max_0,zero_crossing_rate_std_0,zero_crossing_rate_mean_0,zero_crossing_rate_median_0,zero_crossing_rate_kurtosis_0
0,The Weeknd,Blinding Lights,1,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/Blinding Lights - The Weeknd.wav,Blinding Lights,0.325105,0.000000,1.0,...,0.009015,0.001993,0.683537,0.120783,0.000000,0.185714,0.031305,0.075716,0.076916,0.640352
1,Olivia Rodrigo,Good 4 U,1,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/Good 4 U - Olivia Rodrigo.wav,Good 4 U,1.020729,0.000000,1.0,...,-0.001283,-0.001245,2.368123,-0.139140,0.000000,0.274331,0.042506,0.089507,0.096644,0.533097
2,Olivia Rodrigo,Drivers License,1,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/Drivers License - Olivia Rodrig...,Drivers License,1.408063,0.000000,1.0,...,-0.014449,-0.014515,0.351270,1.100266,0.000000,0.248435,0.030082,0.069129,0.065079,3.259196
3,Lil Nas X,Montero (Call Me By Your Name),1,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/Montero (Call Me By Your Name) ...,Montero (Call Me By Your Name),0.995654,0.000000,1.0,...,-0.001599,-0.007845,0.446242,0.525206,0.000000,0.362494,0.052102,0.114976,0.109138,0.934361
4,BTS,Butter,1,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/Butter - BTS.wav,Butter,0.314336,0.000000,1.0,...,0.004915,0.003381,4.878381,0.268373,0.000000,0.250340,0.044437,0.092644,0.090476,1.093109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790,Ne-Yo,Do You,0,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/Do You - Ne-Yo.wav,Do You,0.649463,0.000000,1.0,...,-0.025344,-0.025509,1.378500,1.289102,0.033016,0.303900,0.027651,0.096831,0.094558,5.633785
791,Avril Lavigne,When You're Gone,0,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/When You're Gone - Avril Lavign...,When You're Gone,1.357484,0.001466,1.0,...,0.009152,0.005985,0.361181,3.448056,0.027120,0.538277,0.047306,0.116857,0.119138,27.333917
792,Gwen Stefani,4 In The Morning,0,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/4 In The Morning - Gwen Stefani...,4 In The Morning,0.520500,0.000000,1.0,...,0.008184,-0.005062,1.474136,0.772716,0.000000,0.279184,0.036193,0.105221,0.102313,2.393584
793,Beyonce,Get Me Bodied,0,/home/yamen/Projects/Technocolabs/sample_data/...,/home/yamen/Projects/Technocolabs/Chorus//home...,Extract_Chorus/Get Me Bodied - Beyonce.wav,Get Me Bodied,0.865777,0.000000,1.0,...,0.008661,-0.000451,1.255475,0.859274,0.021088,0.393424,0.059574,0.118995,0.111610,1.050555


In [17]:
final_data.columns # scan for repeated column names 

Index(['Artist', 'Title', 'Label', 'Path', 'Chorus_Path',
       'extracted_chorus_path', 'title', 'chroma_stft_kew_0',
       'chroma_stft_min_0', 'chroma_stft_max_0',
       ...
       'tonnetz_mean_5', 'tonnetz_median_5', 'tonnetz_kurtosis_5',
       'zero_crossing_rate_kew_0', 'zero_crossing_rate_min_0',
       'zero_crossing_rate_max_0', 'zero_crossing_rate_std_0',
       'zero_crossing_rate_mean_0', 'zero_crossing_rate_median_0',
       'zero_crossing_rate_kurtosis_0'],
      dtype='object', length=525)

In [18]:
final_data = final_data.drop(columns=['title'], axis=1) #drop repeated colunm name

In [19]:
final_data.columns

Index(['Artist', 'Title', 'Label', 'Path', 'Chorus_Path',
       'extracted_chorus_path', 'chroma_stft_kew_0', 'chroma_stft_min_0',
       'chroma_stft_max_0', 'chroma_stft_std_0',
       ...
       'tonnetz_mean_5', 'tonnetz_median_5', 'tonnetz_kurtosis_5',
       'zero_crossing_rate_kew_0', 'zero_crossing_rate_min_0',
       'zero_crossing_rate_max_0', 'zero_crossing_rate_std_0',
       'zero_crossing_rate_mean_0', 'zero_crossing_rate_median_0',
       'zero_crossing_rate_kurtosis_0'],
      dtype='object', length=524)

In [20]:
#save table to csv file 
final_data.to_csv('feature_extract.csv', index=False) 